In [45]:
import altair as alt
import pandas as pd

alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

## 0. Data Preprocessing

In [46]:
# Load the dataset
df = pd.read_csv('dpt2020.csv', sep=';')
df.head()

,sexe,preusuel,annais,dpt,nombre
0,1,_PRENOMS_RARES,1900,02,7
1,1,_PRENOMS_RARES,1900,04,9
2,1,_PRENOMS_RARES,1900,05,8
3,1,_PRENOMS_RARES,1900,06,23
4,1,_PRENOMS_RARES,1900,07,9


In [47]:
# Convert annais to integer and nombre to numeric. Discard errors in conversion
df['annais'] = pd.to_numeric(df['annais'], errors='coerce')
df['nombre'] = pd.to_numeric(df['nombre'], errors='coerce')

# Remove rows with missing values
df = df.dropna(subset=['nombre', 'preusuel'])

# Remove _PRENOMS_RARES	
df = df[df['preusuel'] != '_PRENOMS_RARES']

print(df.shape)
df.head()

(3705515, 5)


,sexe,preusuel,annais,dpt,nombre
10882,1,A,NaN,XX,27
10883,1,AADAM,NaN,XX,30
10884,1,AADEL,NaN,XX,56
10885,1,AADIL,1983.0,84,3
10886,1,AADIL,1992.0,92,3


## Visualization 1

In [48]:
# Data preprocessing for viz 1
# Group by year and name, sum the counts across all departments and genders
df_grouped = df.groupby(['annais', 'preusuel'])['nombre'].sum().reset_index()

# Get the top 15 names for each year to make the visualization manageable
def get_top_names_by_year(data, top_n=15):
    result = []
    for year in data['annais'].unique():
        year_data = data[data['annais'] == year].nlargest(top_n, 'nombre')
        result.append(year_data)
    return pd.concat(result, ignore_index=True)

df_top = get_top_names_by_year(df_grouped, top_n=15)


In [49]:

# Create the interactive histogram with time slider
slider = alt.binding_range(min=int(df_top['annais'].min()), 
                          max=int(df_top['annais'].max()), 
                          step=1, 
                          name='Year: ')
select_year = alt.param(bind=slider, value=2000)

# Base chart
base = alt.Chart(df_top).add_params(
    select_year
).transform_filter(
    alt.datum.annais == select_year
).transform_window(
    rank='rank(nombre)',
    sort=[alt.SortField('nombre', order='descending')]
).transform_filter(
    alt.datum.rank <= 15
)

# Main histogram
histogram = base.mark_bar(
    color='steelblue',
    stroke='white',
    strokeWidth=1
).encode(
    x=alt.X('nombre:Q', 
            title='Number of Births',
            scale=alt.Scale(nice=True)),
    y=alt.Y('preusuel:N', 
            title='Name',
            sort=alt.SortField('nombre', order='descending')),
    tooltip=['preusuel:N', 'nombre:Q', 'annais:O']
).properties(
    width=600,
    height=400,
    title=alt.TitleParams(
        text=['Baby Name Popularity by Year', 'Top 15 Names'],
        fontSize=16,
        anchor='start'
    )
)

# Add value labels on bars
text = base.mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=10,
    color='black'
).encode(
    x=alt.X('nombre:Q'),
    y=alt.Y('preusuel:N', sort=alt.SortField('nombre', order='descending')),
    text=alt.Text('nombre:Q', format='.0f')
)

# Combine the chart
chart = (histogram + text).resolve_scale(
    color='independent'
).properties(
    title=alt.TitleParams(
        text='Evolution of Baby Names Over Time',
        subtitle='Use the slider to explore different years',
        fontSize=18,
        subtitleFontSize=12,
        anchor='start',
        offset=20
    )
)

chart.show()

alt.LayerChart(...)

## Visualization 2

In [ ]:
import json

# Data preprocessing for viz 2 - Regional choropleth map
# Group by department and name, sum across years and genders
df_regional = df.groupby(['dpt', 'preusuel'])['nombre'].sum().reset_index()

# Remove the 'XX' department (unknown/other)
df_regional = df_regional[df_regional['dpt'] != 'XX']

# Get top 20 names overall for the dropdown
top_names_regional = df_regional.groupby('preusuel')['nombre'].sum().nlargest(20).index.tolist()

# Load France departments geojson data
with open('departements-version-simplifiee.geojson', 'r') as f:
    geojson_data = json.load(f)

# Create dropdown for name selection
name_selector = alt.binding_select(options=['All'] + top_names_regional, name='Select Name: ')
selected_name_regional = alt.param(bind=name_selector, value='MARIE')

# Base map
choropleth = alt.Chart(alt.InlineData(values=geojson_data['features'])).mark_geoshape(
    fill='lightgray',
    stroke='white',
    strokeWidth=0.5
).encode(
    color=alt.condition(
        selected_name_regional == 'All',
        alt.value('lightgray'),
        alt.Color('nombre:Q',
                 scale=alt.Scale(scheme='blues', type='linear'),
                 title='Number of Births',
                 legend=alt.Legend(orient='right'))
    ),
    tooltip=['properties.nom:N', 'nombre:Q']
).transform_lookup(
    lookup='properties.code',
    from_=alt.LookupData(
        data=df_regional,
        key='dpt',
        fields=['preusuel', 'nombre']
    )
).transform_filter(
    alt.expr(f'datum.preusuel == "{selected_name_regional.name}" || "{selected_name_regional.name}" == "All"')
).add_params(
    selected_name_regional
).properties(
    width=500,
    height=400,
    title=alt.TitleParams(
        text='Regional Distribution of Baby Names in France',
        subtitle='Select a name to see its geographical distribution',
        fontSize=16,
        subtitleFontSize=12,
        anchor='start'
    )
).project(
    type='mercator'
)

choropleth.show()

AttributeError: type object 'expr' has no attribute 'datum'